# Loop 1 LB Feedback Analysis

## Submission Results
- CV Score: 0.8283
- LB Score: 0.7727
- Gap: +0.0556 (CV is optimistic by ~5.6%)

## Key Questions
1. Why is CV so much higher than LB?
2. Is there distribution shift between train and test?
3. What can we learn from the prediction patterns?

In [ ]:
import pandas as pd
import numpy as np

# Load data
train = pd.read_csv('/home/data/train.csv')
test = pd.read_csv('/home/data/test.csv')
submission = pd.read_csv('/home/code/submission_candidates/candidate_000.csv')

print('Train shape:', train.shape)
print('Test shape:', test.shape)
print('Submission shape:', submission.shape)
print('\nSubmission prediction distribution:')
print(submission['Survived'].value_counts(normalize=True))

In [ ]:
# Compare distributions between train and test
print('=== Distribution Comparison ===')
print('\n--- Pclass ---')
print('Train:', train['Pclass'].value_counts(normalize=True).sort_index().values)
print('Test:', test['Pclass'].value_counts(normalize=True).sort_index().values)

print('\n--- Sex ---')
print('Train:', train['Sex'].value_counts(normalize=True))
print('Test:', test['Sex'].value_counts(normalize=True))

print('\n--- Age (mean, std) ---')
print(f'Train: {train["Age"].mean():.2f}, {train["Age"].std():.2f}')
print(f'Test: {test["Age"].mean():.2f}, {test["Age"].std():.2f}')

print('\n--- Fare (mean, std) ---')
print(f'Train: {train["Fare"].mean():.2f}, {train["Fare"].std():.2f}')
print(f'Test: {test["Fare"].mean():.2f}, {test["Fare"].std():.2f}')

In [ ]:
# Check embarked distribution
print('--- Embarked ---')
print('Train:', train['Embarked'].value_counts(normalize=True))
print('Test:', test['Embarked'].value_counts(normalize=True))

# Check SibSp and Parch
print('\n--- SibSp ---')
print('Train:', train['SibSp'].value_counts(normalize=True).head())
print('Test:', test['SibSp'].value_counts(normalize=True).head())

print('\n--- Parch ---')
print('Train:', train['Parch'].value_counts(normalize=True).head())
print('Test:', test['Parch'].value_counts(normalize=True).head())

In [ ]:
# Extract Title and compare
def extract_title(df):
    df = df.copy()
    df['Title'] = df['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)
    title_mapping = {
        'Mlle': 'Miss', 'Ms': 'Miss', 'Mme': 'Mrs',
        'Lady': 'Rare', 'Countess': 'Rare', 'Capt': 'Rare',
        'Col': 'Rare', 'Don': 'Rare', 'Dr': 'Rare',
        'Major': 'Rare', 'Rev': 'Rare', 'Sir': 'Rare',
        'Jonkheer': 'Rare', 'Dona': 'Rare'
    }
    df['Title'] = df['Title'].replace(title_mapping)
    return df

train_t = extract_title(train)
test_t = extract_title(test)

print('--- Title Distribution ---')
print('Train:')
print(train_t['Title'].value_counts(normalize=True))
print('\nTest:')
print(test_t['Title'].value_counts(normalize=True))

In [ ]:
# Analyze survival rates by key features in training data
print('=== Survival Rates in Training Data ===')
print('\n--- By Sex ---')
print(train.groupby('Sex')['Survived'].mean())

print('\n--- By Pclass ---')
print(train.groupby('Pclass')['Survived'].mean())

print('\n--- By Title ---')
print(train_t.groupby('Title')['Survived'].mean())

In [ ]:
# Check if test has different Sex/Pclass combinations
print('=== Sex-Pclass Combinations ===')
train['Sex_Pclass'] = train['Sex'] + '_' + train['Pclass'].astype(str)
test['Sex_Pclass'] = test['Sex'] + '_' + test['Pclass'].astype(str)

print('Train:')
print(train['Sex_Pclass'].value_counts(normalize=True).sort_index())
print('\nTest:')
print(test['Sex_Pclass'].value_counts(normalize=True).sort_index())

In [ ]:
# Calculate expected survival rate based on training patterns
# If test has different distribution, expected rate would differ

# Get survival rates by Sex_Pclass from training
survival_rates = train.groupby('Sex_Pclass')['Survived'].mean()
print('Survival rates by Sex_Pclass (from training):')
print(survival_rates)

# Apply to test distribution
test_dist = test['Sex_Pclass'].value_counts(normalize=True)
expected_survival = sum(survival_rates[sp] * test_dist[sp] for sp in test_dist.index)
print(f'\nExpected survival rate in test (based on train patterns): {expected_survival:.4f}')
print(f'Actual train survival rate: {train["Survived"].mean():.4f}')
print(f'Our prediction survival rate: {submission["Survived"].mean():.4f}')

In [ ]:
# Key insight: The CV-LB gap of 0.0556 is substantial
# Possible causes:
# 1. Overfitting to training data patterns
# 2. Distribution shift in test set
# 3. Small test set variance (418 samples)

# The LB score of 0.7727 means we got ~323 correct out of 418
# CV score of 0.8283 would mean ~738 correct out of 891 in training

print('=== Gap Analysis ===')
print(f'CV accuracy: 0.8283 -> ~{int(0.8283 * 891)} correct out of 891')
print(f'LB accuracy: 0.7727 -> ~{int(0.7727 * 418)} correct out of 418')
print(f'Gap: 5.56 percentage points')
print('\nThis gap suggests our model may be overfitting to training patterns')
print('or there is distribution shift between train and test.')